In [1]:
import sys
sys.path.append('../../../')
from pprint import pprint
import time
import pickle
import importlib
import numpy as np

In [2]:
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
from lib_grover import make_hit_list
from libs_qrem import NationEtalFilter

In [3]:
nbit = 10 # change this value to get discretized result closer to analytical results # 3
max_iter = 10

In [4]:
with open("pkls/raw_hist_list_list.pkl", "rb") as f:
    raw_hist_list_list = pickle.load(f)
with open("pkls/meas_fitters.pkl", "rb") as f:
    meas_fitters = pickle.load(f)

In [5]:
nation_bicgstab_mitigator_list = []
for i in range(0, max_iter):
    nation_bicgstab_mitigator_list.append(NationEtalFilter(nbit + 1, meas_fitters[i].cal_matrices))
    if i % 10 == 0:
        print("size", i, "finished")
print("length of nation_bicgstab_mitigator_list: ", len(nation_bicgstab_mitigator_list))

size 0 finished
length of nation_bicgstab_mitigator_list:  10


In [6]:
def get_info(mitigator):
    return {"exact_one_norm_of_inv_reduced_A": mitigator.exact_one_norm_of_inv_reduced_A(),
            "iterative_one_norm_of_inv_reduced_A": mitigator.iterative_one_norm_of_inv_reduced_A("iterative"),
            "mitigated_hist": mitigator.mitigated_hist(),
            "times": mitigator.times(),
            "mitigation_stddev": mitigator.mitigation_stddev(norm_type = "exact"),
            }

In [7]:
def make_hit_list(hist_list):
    hit_list = []
    for k in range(len(hist_list)):
        hits = 0
        if hist_list[k]:
            hits = hist_list[k].get("0" * len(list(hist_list[k].keys())[0]), 0)
        hit_list.append(hits)
    return hit_list

In [8]:
nation_bicgstab_info_list_list = []
nation_bicgstab_hit_list_list = []
for i, raw_hist_list in enumerate(raw_hist_list_list):
    nation_bicgstab_info_list = []
    nation_bicgstab_hist_list = []
    for raw_hist in raw_hist_list:
        nation_bicgstab_hist_list.append( nation_bicgstab_mitigator_list[i].apply(raw_hist) )
        nation_bicgstab_info_list.append( get_info(nation_bicgstab_mitigator_list[i]) )
    nation_bicgstab_hit_list_list.append( make_hit_list(nation_bicgstab_hist_list) )
    nation_bicgstab_info_list_list.append( nation_bicgstab_info_list )
    print("iter", i, "finished")

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished


In [9]:
with open("pkls/nation_bicgstab_info_list_list.pkl", "wb") as f:
    pickle.dump(nation_bicgstab_info_list_list, f)
with open("pkls/nation_bicgstab_hit_list_list.pkl", "wb") as f:
    pickle.dump(nation_bicgstab_hit_list_list, f)